In [114]:
import csv
import pandas as pd

In [115]:
full_dataset = pd.read_csv('IncompleteDataSet.csv')

mismatch_sarah = []
mismatch_laura = []
sarahs_mismatch = 0
laura_mismatch = 0
for index, row in full_dataset.iterrows():
    try:
        if row['title_x'].lower() != row['title_y'].lower():
            
            if row['imdb_id'] in sarahs_ids:
                sarahs_mismatch += 1
                mismatch_sarah.append([row['imdb_id'],row['title_x'],row['title_y']])
            else:
                laura_mismatch += 1
                mismatch_laura.append([row['imdb_id'],row['title_x'],row['title_y']])
    except:
        if row['title_x'] != row['title_y']:
            if row['imdb_id'] in sarahs_ids:
                sarahs_mismatch += 1
                mismatch_sarah.append([row['imdb_id'],row['title_x'],row['title_y']])
            else:
                laura_mismatch += 1
                mismatch_laura.append([row['imdb_id'],row['title_x'],row['title_y']])


# Perform Bag of Words 

In [116]:
##bag of words on title
from sklearn.feature_extraction.text import CountVectorizer

titles = []
for i in full_dataset['title_x']:
    try:
        title_words.append(i.decode('utf-8'))
        titles.append(i)
    except:
        titles.append(0)
        
#remove movies that have non-ascii titles, I ran into lots of problems with this so we
#decided to just delete these movies
full_dataset['title_x'] = titles
full_dataset = full_dataset[full_dataset['title_x'] != 0] 

title_words = []
for i in full_dataset['title_x']:
    title_words.append(i)

#run bag of words and keep 100 features 
vectorizer = CountVectorizer(stop_words = 'english', max_features = 100)
data_features = vectorizer.fit_transform(title_words).todense()

In [117]:
#give informative column names 
feature_names = ['title_'+i.encode('utf-8') for i in vectorizer.get_feature_names()]

for i in range(len(feature_names)):
    full_dataset[feature_names[i]] = data_features[:,i]
del full_dataset['title_x']

## Bag of Words on Overviews

In [118]:
count = 0
overview_words = []
for i in full_dataset['overview']:
    try:
        overview_words.append(i.decode('utf-8'))
    except:
        overview_words.append('')

vectorizer = CountVectorizer(stop_words = 'english', max_features = 100)
overview_data_features = vectorizer.fit_transform(overview_words).todense()

In [119]:
overview_feature_names = ['overview_'+i.encode('utf-8') for i in vectorizer.get_feature_names()]
for i in range(len(overview_feature_names)):
    full_dataset[overview_feature_names[i]] = overview_data_features[:,i]
del full_dataset['overview']
full_dataset.to_csv('dataset_bag_of_words.csv', index = False)

## Deal With Genres

In [120]:
current_genres =  ['Action', 'Adventure', 'Adult', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Short', 'Thriller', 'War', 'Western', 'Talk-Show', 'News', 'Game-Show', 'Reality-TV', 'History', 'Sport']
for i in current_genres:
    del full_dataset[i]

In [121]:
acceptable_genres = ['Action','Adventure','Animation','Comedy','Crime','Documentary','Drama','Family','Fantasy','History','Horror','Mystery','Romance','Science Fiction','Thriller','War','Western']
new_genres = []
for i in acceptable_genres:
    new_genres.append([])

for index, row in full_dataset.iterrows():
    
    #x genres
    gen_x = row['genres_x']
    genre_x = []
    try:
        g1 = gen_x.replace("]", "")
        g1 = g1.replace("[", "")
        g1 = g1.split(',')
        for c in g1:
            c = c[c.index('\''):]
            c = c.strip('\'')
            genre_x.append(c)
            if c == 'Sci-Fi':
                c = 'Science Fiction'
    except:
        g1 = []
    
    #y genres
    gen_y = row['genres_y']
    genre_y = []
    try:
        g2 = gen_y.replace("]", "")
        g2 = g2.replace("[", "")
        g2 = g2.split(',')
        for c in g2:
            c = c[c.index('\''):]
            c = c.strip('\'')
            if c == 'Sci-Fi':
                c = 'Science Fiction'
            genre_y.append(c)
    except:
        g2 = []
    
    
    intersection =  list(set(genre_y).intersection(genre_x))
    
    if intersection == []:
        for g in range(len(acceptable_genres)):
            new_genres[g].append('Drop')
        
    else:
        for g in range(len(acceptable_genres)):
            if acceptable_genres[g] in intersection:
                new_genres[g].append(1)
            else:
                new_genres[g].append(0)
    

#delete rows with no genre information
for i in range(len(acceptable_genres)):
    full_dataset[acceptable_genres[i]] = new_genres[i]
    
full_dataset_new = full_dataset[full_dataset['Adventure'] != 'Drop']

#delte genre columns
del full_dataset_new['genres_x']
del full_dataset_new['genres_y']

full_dataset_new.to_csv('dataset_final.csv', index = False)

In [122]:
## delete id, title_y, imdb_id, movie_id, Unnamed: 0
import numpy as np
del full_dataset_new['id']
del full_dataset_new['title_y']
del full_dataset_new['imdb_id']
del full_dataset_new['movie_id']
del full_dataset_new['Unnamed: 0']

full_dataset_new.replace(r'\s+', np.nan, regex=True)

full_dataset_new.to_csv('dataset_final.csv', index = False, na_rep = np.nan)

         revenue  vote_count  popularity     budget  vote_average  runtime  \
0              0           1    0.000155          0           6.0     65.0   
1              0           4    0.003423          0           5.9      6.0   
3              0           0    0.000215          0           0.0    300.0   
4              0           2    0.110808          0           5.0     92.0   
5              0           1    0.000502          0           8.0     86.0   
6              0           2    0.007932          0           5.5      0.0   
7              0           3    0.000729          0           7.5     90.0   
8              0           0    0.000747          0           0.0    130.0   
9      392000694        4961    8.056843  100000000           7.9    180.0   
10             0           6    0.135404          0           6.0    100.0   
13             0           0    0.000515          0           0.0     88.0   
14             0           1    0.034704          0           7.